<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/daytrading_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving 1 386 매매내역_(신이준-선물)_2021-07-31.xls to 1 386 매매내역_(신이준-선물)_2021-07-31.xls


In [57]:
df = pd.read_html(uploaded['1 386 매매내역_(신이준-선물)_2021-07-31.xls'])[0]

In [58]:
import datetime

df.columns = df.iloc[0]
df = df.iloc[1:]

df['거래일자'] = list(map(lambda x: datetime.datetime.strptime(x, '%m/%d %H:%M'), df['거래일자']))
df['매수/매도가'], df['수량'], df['매매금액'] = df['매수/매도가'].astype(float), df['수량'].astype(int), df['매매금액'].astype(int)

In [59]:
df = df[::-1].reset_index(drop=True)
df

,거래일자,매매구분,종목명,매수/매도가,수량,매매금액
0,1900-06-14 08:30:00,매도,라온시큐어,6070.0,8,48560
1,1900-06-14 08:30:00,매도,라온시큐어,6070.0,1222,7417540
2,1900-06-14 08:30:00,매도,라온시큐어,6070.0,860,5220200
3,1900-06-14 08:30:00,매도,라온시큐어,6070.0,650,3945500
4,1900-06-14 08:30:00,매도,라온시큐어,6070.0,329,1997030
...,...,...,...,...,...,...
3409,1900-07-29 16:12:00,매수,나노캠텍,2460.0,7000,17220000
3410,1900-07-29 16:20:00,매도,나노캠텍,2350.0,7000,16450000
3411,1900-07-29 16:30:00,매도,우리바이오,5350.0,4622,24727700
3412,1900-07-29 16:50:00,매수,아즈텍WB,4175.0,10000,41750000
